#### Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import seaborn as sns

from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timedelta

from sklearn.impute import SimpleImputer

In [ ]:
## CHANGE DIRECTORY TO WHERE THE RAW DATA IS LOCATED
os.chdir("/anonymized_path/sakshikumar/Documents/UCD/04. SP 2025/464 Practicum/KWSM finale/FINAL FILES")

### AHrefs Data Processing

In [ ]:
ah = pd.read_excel('organickeywords.xlsx')
ah.head()

### Processing and Augmenting AHrefs Data

In [ ]:
# filter out the blog web
ah = ah[~ah['Current URL'].str.startswith("https://kwsmdigital.com/blog/")].reset_index(drop = True)

# create an intent metrics
ah[['Navigational', 'Informational', 'Commercial', 'Transactional']] = ah[
    ['Navigational', 'Informational', 'Commercial', 'Transactional']
].astype(int)

# High-intent
ah["intent"] = ((ah["Commercial"] == 1) | (ah["Transactional"] == 1)).astype(int)

# Number of words
ah['Word_Count'] = ah['Keyword'].apply(lambda x: len(x.split()))

# SERP_feature
ah['Has_SERP_Feature'] = ah['SERP features'].apply(lambda x: 1 if pd.notna(x) and x.strip() != '' else 0)

# Extract all data
ah = ah[['Keyword', 'Current URL', 'KD', 'Volume','CPC', 'Organic traffic', 'intent', 'Navigational', 'Informational',	'Commercial', 'Transactional', 'Word_Count', 	'Has_SERP_Feature']].rename(columns = {'Current URL':'URL', 'Keyword':'keyword'})

ah

In [ ]:
# Define the target columns expected in AHREFS_DATA.csv
target_columns = [
    'keyword', 'URL', 'KD', 'Volume', 'CPC', 'Organic traffic',
    'intent', 'Navigational', 'Informational', 'Commercial', 'Transactional',
    'Word_Count', 'Has_SERP_Feature', 'Clicks', 'Impressions', 'CTR', 'Position'
]

# Initialize missing columns with default values
# Assuming default for Clicks, Impressions, CTR, and Position is 0 or NaN if appropriate
missing_defaults = {
    'Clicks': 0,
    'Impressions': 0,
    'CTR': 0.0,
    'Position': 0.0
}

for col in target_columns:
    if col not in ah.columns:
        default_value = missing_defaults.get(col, None)
        ah[col] = default_value

# Reorder the columns to match the expected AHREFS_DATA.csv format
ahrefs_processed = ah[target_columns]

ahrefs_processed.shape

# Save the new dataframe
# ahrefs_processed.to_csv('AHREFS_DATA.csv', index=False)

# print("New dataset saved as 'AHREFS_DATA.csv'")

### GSC Data Processing

#### Fetching data via API

In [ ]:
# 1️⃣ Authenticate API
SERVICE_ACCOUNT_FILE = "client_secret_gsc.json"  # Path to your JSON key file
SCOPES = ["https://www.googleapis.com/auth/webmasters.readonly"]

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
service = build("searchconsole", "v1", credentials=credentials)

# 2️⃣ Set API request parameters
REQUEST_TEMPLATE = {
    "startDate": "2024-01-24",  # Last 12 months
    "endDate": "2025-01-24",
    "dimensions": ["query", "page"],  # Get keyword + URL data
    "rowLimit": 25000  # GSC API limit: max 25,000 rows
}

# 3️⃣ Handle API pagination
def fetch_gsc_data():
    all_data = []
    start_row = 0  # Start from row 0

    while True:
        request = REQUEST_TEMPLATE.copy()
        request["startRow"] = start_row  # Pagination parameter

        response = service.searchanalytics().query(siteUrl="https://kwsmdigital.com/", body=request).execute()

        if "rows" in response:
            for row in response["rows"]:
                query, page = row["keys"]
                clicks = row.get("clicks", 0)
                impressions = row.get("impressions", 0)
                ctr = row.get("ctr", 0)
                position = row.get("position", 0)

                # 🔹 Filter out pages that start with "https://kwsmdigital.com/blog"
                if not page.startswith("https://kwsmdigital.com/blog"):
                    all_data.append([query, page, clicks, impressions, ctr, position])

            start_row += len(response["rows"])  # Move to next page
            print(f"✅ Retrieved {len(response['rows'])} rows, total {len(all_data)} rows (after filtering blog pages)...")

            if len(response["rows"]) < 25000:
                break  # If fewer than 25,000 rows, all data has been fetched
        else:
            print("⚠️ No more data returned.")
            break

    return all_data


In [ ]:
# 3. Function to fetch data from GSC
# This function retrieves data from Google Search Console, filtering out blog pages.
def fetch_gsc_data():
    all_data = []
    start_row = 0  # Start from row 0

    while True:
        request = REQUEST_TEMPLATE.copy()
        request["startRow"] = start_row  # Pagination parameter

        response = service.searchanalytics().query(siteUrl="https://kwsmdigital.com/", body=request).execute()

        if "rows" in response:
            for row in response["rows"]:
                query, page = row["keys"]
                clicks = row.get("clicks", 0)
                impressions = row.get("impressions", 0)
                ctr = row.get("ctr", 0)
                position = row.get("position", 0)

                # 🔹 Filter out pages that start with "https://kwsmdigital.com/blog"
                if not page.startswith("https://kwsmdigital.com/blog"):
                    all_data.append([query, page, clicks, impressions, ctr, position])

            start_row += len(response["rows"])  # Move to next page
            print(f"✅ Retrieved {len(response['rows'])} rows, total {len(all_data)} rows (after filtering blog pages)...")

            if len(response["rows"]) < 25000:
                break  # If fewer than 25,000 rows, all data has been fetched
        else:
            print("⚠️ No more data returned.")
            break

    return all_data

In [ ]:
# 4. Run API query
data = fetch_gsc_data()


In [ ]:
# Step 1: Convert to DataFrame
df_gsc = pd.DataFrame(data, columns=["Keyword", "URL", "Clicks", "Impressions", "CTR", "Position"])

df_gsc.shape

In [ ]:
df_gsc.to_csv('temp_gsc.csv', index=False)

In [ ]:
# 2. Remove Keyword column
df_gsc = df_gsc[["URL", "Clicks", "Impressions", "CTR", "Position"]]

# 3. Filter for meaningful data (optional but good practice)
df_gsc = df_gsc[(df_gsc['Clicks'] > 0) & (df_gsc['Impressions'] > 0)]
# Step 4: Sort by Clicks descending
gsc_aggregated = df_gsc.sort_values(by='Clicks', ascending=False)

# Step 5: Optional - keep top N if needed
gsc_aggregated = gsc_aggregated.head(155)

In [ ]:
# Step 6: Save to CSV
## 	•	Due to aggregation, the resulting dataset structure may differ slightly from prior manually curated datasets.
##	•	This process ensures full reproducibility for future GSC data pulls.
gsc_aggregated.to_csv('GSC_DATA_v2.csv', index=False)

### Merging All 3 Data

#### Load GA4 Data

In [ ]:
os.getcwd()

In [ ]:
metr_1 = pd.read_csv("GA4_DATA.csv", skiprows=6)
metr_1.reset_index(inplace=True)
metr_1

In [ ]:
metr_1.drop(index=0, inplace=True)
metr_1 = metr_1.iloc[:,:-1]
metr_1.columns = ['Landing page', 'Sessions', 'Active users', 'New users',
       'Returning users', 'Total users', 'Event count',
       'Average engagement time per session', 'Bounce rate',
       'Views per session', 'Engaged sessions per active user']
metr_1['Landing page'] = "https://kwsmdigital.com" + metr_1['Landing page']
metr_1.rename(columns = {"Landing page": "URL"}, inplace=True)
metr_1

#### Load GSC Data

In [ ]:
gsc_page = pd.read_csv("GSC_DATA.csv")
gsc_page['URL'] = gsc_page['URL'].str.rstrip('/')
gsc_page

In [ ]:
gsc_page.shape

#### Load AHREFS Data

In [ ]:
ahref = pd.read_csv("AHREFS_DATA.csv")
ahref['URL'] = ahref['URL'].str.rstrip('/')
ahref.rename(columns={"keyword":"Keyword"}, inplace=True)
ahref = ahref.iloc[:, :-4]
ahref.head()

In [ ]:
ahref.shape

#### Load Mapping Table to help Merge

In [ ]:
map_tb = pd.read_csv("MAPPING_TABLE.csv")
map_tb['URL'] = map_tb['URL'].str.rstrip('/')
map_tb.drop(columns='Source', inplace=True)
map_tb

#### Merge all 3 Data

In [ ]:
merge1 = pd.merge(metr_1, gsc_page, on="URL", how="inner")
merge1

In [ ]:
valid_mappings = pd.merge(
    merge1,
    map_tb,
    on='URL',
    how='inner'
)
valid_mappings

In [ ]:
merge2_exploded = valid_mappings.explode('Keyword')
merge2_exploded

In [ ]:
final_merged = pd.merge(
    merge2_exploded,
    ahref,
    on=['URL', 'Keyword'],
    how='inner'
)
print(final_merged.shape)
final_merged.head()

#### Display Final Merged Dataset

In [ ]:
final_merged.head()

#### Checks to Ensure No Missing Values and Data Imputation

In [ ]:
numeric_cols = final_merged.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='median')
final_merged[numeric_cols] = imputer.fit_transform(final_merged[numeric_cols])
final_merged.isna().sum().sum()

#### Previewing Dataset Information

In [ ]:
print(final_merged.describe().T)

#### Checking Correlation of all Variables

In [ ]:
plt.figure(figsize=(20, 15))
correlation = final_merged.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".2f")
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

In [ ]:
def get_highly_correlated_variables(df, threshold):
    corr_matrix = df.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    highly_correlated = upper_triangle.stack().sort_values(ascending=False)
    highly_correlated = highly_correlated[highly_correlated > threshold]
    
    return highly_correlated

highly_correlated_df = get_highly_correlated_variables(correlation, 0.7)
print(highly_correlated_df)

#### Feature Engineering for Modelling Input

In [ ]:
final_merged['Commercial_Trans_Ratio'] = final_merged['Commercial'] / (final_merged['Transactional'] + 1)  # Add 1 to avoid division by zero
final_merged['Intent_Value_Score'] = final_merged['Commercial'] * 0.4 + final_merged['Transactional'] * 0.6  # Weighting based on conversion value
final_merged['Intent_KD_Ratio'] = final_merged['intent'] / (final_merged['KD'] + 1)  # Value-to-difficulty ratio
final_merged['Volume_Intent_Interaction'] = final_merged['Volume'] * final_merged['intent']
final_merged['Position_Inverse'] = 100 / (final_merged['Position'] + 1)  # Transform position to a "higher is better" metric

#### Export to CSV

In [ ]:
final_merged.to_csv("dataset.csv", index=False)